In [158]:
import csv
import os
import requests

# Projekt

## 1.Pridobivanje podatkov

Spremenljivke

In [159]:
#url
characters_malaz_url = "https://en.wikipedia.org/wiki/List_of_Malazan_Book_of_the_Fallen_characters"

#html strani
page_html = 'data.html'

#csv datoteka obdelanih podatkov
csv_characters = 'osebe.csv'
csv_knjige = 'knjige.csv'

#mapa, v katero spravimo neobdelane (html) podatke
directory_neobdelani = 'podatki-neobdelani'

#mapa, v katero sppravimo obdelane (csv) podatke
directory_obdelani = 'podatki-obdelani'

Funkcije

In [160]:
#pridobivanje podatkov iz spleta
def get_data_from_page(url):
    try:
        response = requests.get(url)
        if response.status_code == 200: #vse ok
            return response.text
        else: 
            raise ValueError(f'Napačna koda: {response.status_code}')
    except Exception: #koda pri napaki
        print(f'Nalaganje strani {url} ni uspelo.')

#shranjevanje podatkov v file
def save_string_to_file(text, directory, filename):
    if directory != "": #ustvari mapo, da sigurno obstaja
        os.makedirs(directory, exist_ok=True)
    path = os.path.join(directory, filename)
    #odpreš file in v njega vpišeš datoteko
    with open(path, 'w', encoding='utf-8') as file_out:
        file_out.write(text)
    return None

In [161]:
#skupaj pridobivanje podatkov
def save_frontpage(page, directory, filename):
    vsebina = get_data_from_page(page)
    if vsebina != None:
        save_string_to_file(vsebina, directory, filename)
    

In [162]:
save_frontpage(characters_malaz_url, directory_neobdelani, page_html)

## 2. Ekstrakcija podatkov - osebe

Funkcije

In [163]:
#spremeni html datoteko v string
def read_file_to_string(directory, filename):
    path = os.path.join(directory, filename)
    with open(path, 'r', encoding='utf-8') as file_out:
        return file_out.read()

In [164]:
#pridobimo bloke z informacijami
import re
def page_to_list_of_names(page_content):
    vzorec = r"<td><b>.+?</td></tr>"
    return re.findall(vzorec, page_content, flags=re.DOTALL) 

In [165]:
#preverjanje vzorca
text = read_file_to_string(directory_neobdelani, page_html)
characters = page_to_list_of_names(text)
print(len(characters))

694


In [166]:
#bloke spremeni v slovarje
def get_dict_from_character_block(block):
    vzorec_ime = r"<td><b>(.+)</b>"
    
    vzorec_opis= r"</b>\n</td>(.+?)\n</td>"
   
    vzorec_pojavljanja = r'<td>.+</td>.+<td>(.*)</td></tr>'
    

    character = {}

    character['ime'] = re.search(vzorec_ime, block).group(1)
    character['pojavljanje'] = re.search(vzorec_pojavljanja, block, flags=re.DOTALL).group(1)
    character['opis'] = re.search(vzorec_opis, block, flags=re.DOTALL).group(1)
    
    #umikanje hiperpovezav, ležečih pisav ter znakov za nove vrstice
    character['opis'] = re.sub('<.*?>', '', character['opis'])
    character['opis'] = re.sub('\n', '', character['opis'])
    character['pojavljanje'] = re.sub('<.*?>', '', character['pojavljanje'])
    character['pojavljanje'] = re.sub('\n', '', character['pojavljanje'])


    return character

In [167]:
#preverjanje
for character in characters:
    print(get_dict_from_character_block(character))

{'ime': "A'Karonys", 'pojavljanje': 'GotM', 'opis': 'High Mage of the Malazan Empire'}
{'ime': 'Abide', 'pojavljanje': 'CG', 'opis': 'Pure Forkrul Assail, lawful inquisitor, mystic'}
{'ime': 'Able', 'pojavljanje': 'HoC', 'opis': 'Soldier in the Bonehunters'}
{'ime': 'Abrastal', 'pojavljanje': 'DoD, CG', 'opis': 'Queen of Bolkando'}
{'ime': 'Absi', 'pojavljanje': 'DoD, CG', 'opis': "Son of Onos T'oolan (Tool) and Hetan"}
{'ime': 'Agayla', 'pojavljanje': 'BH, NoK, SW', 'opis': 'A mysterious mage on Malaz Isle with connections to many other people'}
{'ime': 'Ahlrada Ahn', 'pojavljanje': 'MT, BH', 'opis': 'Tiste Andii spy among the Tiste Edur'}
{'ime': 'Aimless', 'pojavljanje': 'MoI', 'opis': 'Corporal in the Bridgeburners'}
{'ime': 'Aloft', 'pojavljanje': 'CG', 'opis': 'Pure Forkrul Assail, lawful inquisitor'}
{'ime': 'Amby Bole', 'pojavljanje': 'TH, DoD, CG', 'opis': 'Retired Mott Irregular and Trygalle Trade Guild shareholder; brother to Jula Bole'}
{'ime': 'Amiss', 'pojavljanje': 'CG',

In [168]:
#združeno pridobivanje v bloke in pretvorba v slovarje
def characters_from_file(filename, directory):
    prebrano = read_file_to_string(directory, filename)
    bloki = page_to_list_of_names(prebrano)
    osebe = []
    for i in bloki:
        try:
            osebe.append(get_dict_from_character_block(i))
        except Exception:
            continue
    return osebe

In [169]:
#testiranje
characters_from_file(page_html, directory_neobdelani)

[{'ime': "A'Karonys",
  'pojavljanje': 'GotM',
  'opis': 'High Mage of the Malazan Empire'},
 {'ime': 'Abide',
  'pojavljanje': 'CG',
  'opis': 'Pure Forkrul Assail, lawful inquisitor, mystic'},
 {'ime': 'Able', 'pojavljanje': 'HoC', 'opis': 'Soldier in the Bonehunters'},
 {'ime': 'Abrastal', 'pojavljanje': 'DoD, CG', 'opis': 'Queen of Bolkando'},
 {'ime': 'Absi',
  'pojavljanje': 'DoD, CG',
  'opis': "Son of Onos T'oolan (Tool) and Hetan"},
 {'ime': 'Agayla',
  'pojavljanje': 'BH, NoK, SW',
  'opis': 'A mysterious mage on Malaz Isle with connections to many other people'},
 {'ime': 'Ahlrada Ahn',
  'pojavljanje': 'MT, BH',
  'opis': 'Tiste Andii spy among the Tiste Edur'},
 {'ime': 'Aimless',
  'pojavljanje': 'MoI',
  'opis': 'Corporal in the Bridgeburners'},
 {'ime': 'Aloft',
  'pojavljanje': 'CG',
  'opis': 'Pure Forkrul Assail, lawful inquisitor'},
 {'ime': 'Amby Bole',
  'pojavljanje': 'TH, DoD, CG',
  'opis': 'Retired Mott Irregular and Trygalle Trade Guild shareholder; brother t

## 3. Ekstrakcija podatkov - kratice za knjige

Funkcije (skoraj enak postopek kot pri ekstrakciji za osebe)

In [170]:
#spremeni html datoteko v string
#že imamo kot read_file_to_string

In [171]:
#pridobimo bloke z informacijami
def page_to_list_of_books(page_content):
    vzorec = r"-?\n?<i>[^<].+?[^\s]</a></i>\)"
    return re.findall(vzorec, page_content, flags=re.DOTALL) 

In [172]:
#preverjanje vzorca
text = read_file_to_string(directory_neobdelani, page_html)
books = page_to_list_of_books(text)
print(len(books))

16


In [173]:
#bloke spremeni v slovarje
def get_dict_from_books_block(block):
    vzorec_kratica = r"<i>(.+)</i>\s"
    
    vzorec_ime = r"href=.+>(.+?)</a></i>"   
    
    books = {}

    books['ime'] = re.search(vzorec_ime, block).group(1)
    books['kratica'] = re.search(vzorec_kratica, block).group(1)
    
    #umikanje hiperpovezav, ležečih pisav ter znakov za nove vrstice

    return books

In [174]:
#preverjanje
for book in books:
    print(get_dict_from_books_block(book))

{'ime': 'Gardens of the Moon', 'kratica': 'GotM'}
{'ime': 'Deadhouse Gates', 'kratica': 'DG'}
{'ime': 'Memories of Ice', 'kratica': 'MoI'}
{'ime': 'House of Chains', 'kratica': 'HoC'}
{'ime': 'Midnight Tides', 'kratica': 'MT'}
{'ime': 'The Bonehunters', 'kratica': 'BH'}
{'ime': "Reaper's Gale", 'kratica': 'RG'}
{'ime': 'Toll the Hounds', 'kratica': 'TH'}
{'ime': 'Dust of Dreams', 'kratica': 'DoD'}
{'ime': 'The Crippled God', 'kratica': 'CG'}
{'ime': 'Night of Knives', 'kratica': 'NoK'}
{'ime': 'Return of the Crimson Guard', 'kratica': 'RotCG'}
{'ime': 'Stonewielder', 'kratica': 'SW'}
{'ime': 'Blood and Bone', 'kratica': 'BB'}
{'ime': 'Orb Sceptre Throne', 'kratica': 'OST'}
{'ime': 'Assail', 'kratica': 'As'}


In [175]:
#združeno pridobivanje v bloke in pretvorba v slovarje
def books_from_file(filename, directory):
    prebrano = read_file_to_string(directory, filename)
    bloki = page_to_list_of_books(prebrano)
    knjige = []
    for i in bloki:
        try:
            knjige.append(get_dict_from_books_block(i))
        except Exception:
            continue
    return knjige

In [176]:
#testiranje
books_from_file(page_html, directory_neobdelani)

[{'ime': 'Gardens of the Moon', 'kratica': 'GotM'},
 {'ime': 'Deadhouse Gates', 'kratica': 'DG'},
 {'ime': 'Memories of Ice', 'kratica': 'MoI'},
 {'ime': 'House of Chains', 'kratica': 'HoC'},
 {'ime': 'Midnight Tides', 'kratica': 'MT'},
 {'ime': 'The Bonehunters', 'kratica': 'BH'},
 {'ime': "Reaper's Gale", 'kratica': 'RG'},
 {'ime': 'Toll the Hounds', 'kratica': 'TH'},
 {'ime': 'Dust of Dreams', 'kratica': 'DoD'},
 {'ime': 'The Crippled God', 'kratica': 'CG'},
 {'ime': 'Night of Knives', 'kratica': 'NoK'},
 {'ime': 'Return of the Crimson Guard', 'kratica': 'RotCG'},
 {'ime': 'Stonewielder', 'kratica': 'SW'},
 {'ime': 'Blood and Bone', 'kratica': 'BB'},
 {'ime': 'Orb Sceptre Throne', 'kratica': 'OST'},
 {'ime': 'Assail', 'kratica': 'As'}]

## 4.Shranjevanje podatkov

Funkcije

In [177]:
#zapisovanje v csv
def write_csv(fieldnames, rows, directory, filename):
    if directory != '':
        os.makedirs(directory, exist_ok=True)
    path = os.path.join(directory, filename)
    with open(path, 'w', encoding='utf-8', newline='') as csv_file: #newline prepreči razmake med vrsticami
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader() #zgornja vrstica
        for row in rows:
            writer.writerow(row)
    return

In [178]:
#zapisujemo karakterje v csv datoteko
def write_characters_to_csv(characters, directory, filename):
    # Stavek assert preveri da zahteva velja
    # Če drži se program normalno izvaja, drugače pa sproži napako
    # Prednost je v tem, da ga lahko pod določenimi pogoji izklopimo v
    # produkcijskem okolju
    assert characters and (all(j.keys() == characters[0].keys() for j in characters))
    
    fieldnames = ['ime', 'pojavljanje', 'opis']
    
    write_csv(fieldnames, characters, directory, filename)

In [179]:
#zapisujemo knjige v csv datoteko
def write_books_to_csv(books, directory, filename):
    # Stavek assert preveri da zahteva velja
    # Če drži se program normalno izvaja, drugače pa sproži napako
    # Prednost je v tem, da ga lahko pod določenimi pogoji izklopimo v
    # produkcijskem okolju
    assert books and (all(j.keys() == books[0].keys() for j in books))
    
    fieldnames = ['ime', 'kratica']
    
    write_csv(fieldnames, books, directory, filename)

## 5.Skupaj

Funkcije

In [180]:
#krovna funkcija, ki zajame vso kodo, napisano zgoraj
def zagon(ponovno_pridobivanje_podatov=True, ponovna_obdelava_characters=True, ponovna_obdelava_books=True):
    # 1. Najprej v lokalno datoteko shranimo glavno stran
    pot_html = os.path.join(directory_neobdelani, page_html)
    if ponovno_pridobivanje_podatov:
        # če zahtevamo ponovno nalaganje ali pa html_datoteka
        # še ne obstaja, jo naložimo s spleta
        save_frontpage(characters_malaz_url, directory_neobdelani, page_html)
    
    pot_csv = os.path.join(directory_obdelani, csv_characters)
    # 2. in 3.: obdelava html-ja in shranjevanje v csv
    if ponovna_obdelava_characters:
        # če zahtevamo ponovno obdelavo ali pa csv_datoteka
        # še ne obstaja, jo ustvarimo (ponovno)
        vsi_slovarji = characters_from_file(page_html, directory_neobdelani)
        write_characters_to_csv(vsi_slovarji, directory_obdelani, csv_characters)
   
    
    if ponovna_obdelava_books:
        # če zahtevamo ponovno obdelavo ali pa csv_datoteka
        # še ne obstaja, jo ustvarimo (ponovno)
        vsi_slovarji = books_from_file(page_html, directory_neobdelani)
        write_books_to_csv(vsi_slovarji, directory_obdelani, csv_knjige)
    

In [181]:
zagon(True, True, True)